In [1]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
from google.colab import data_table as dt

from google.auth import default
creds, _ = default()

import pandas_gbq

from datetime import datetime
import pytz

import gspread
gc = gspread.authorize(creds)

import pandas as pd
from datetime import date
import time

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import output

from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

import pandas as pd
from pandas.plotting import autocorrelation_plot

# elimina os warnings das bibliotecas
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Obter o tempo inicial
tempo_inicial = time.time()
#Acessando a sandbox de canais
project = 'ssbx-revops' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
dt.enable_dataframe_formatter()
auth.authenticate_user()

In [3]:
# Para usar no versionamento dos arquivos
# Fuso horário de Brasília
fuso_brasilia = pytz.timezone('America/Sao_Paulo')
# Data e hora atual do sistema
data_hora_utc = datetime.utcnow()

In [4]:
# Converte a data e hora para o fuso horário de Brasília
data_hora_brasilia = data_hora_utc.replace(tzinfo=pytz.utc).astimezone(fuso_brasilia)

data_hora_atual = datetime.now()

# Convertendo para string no formato desejado
data_hora_string = data_hora_brasilia.strftime('%Y-%m-%d %H:%M:%S')  # Formato ano-mês-dia hora:minuto:segundo

from google.colab import drive
drive.mount('/content/drive')
#Especifico o caminho dos arquivos
data_path = '/content/drive/MyDrive/Forecast/History/Indireto1405'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import drive
import pickle
drive.mount('/content/drive')

# Especificar o caminho dos arquivos no Google Drive
historico_file = '/content/drive/MyDrive/Forecast/historico_projecoes.pkl'

try:
    with open(historico_file, 'rb') as f:
        df_historico = pickle.load(f)
    print("Histórico carregado com sucesso.")
except FileNotFoundError:
    print("Histórico não encontrado. Começando do zero.")
    df_historico = pd.DataFrame()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Histórico carregado com sucesso.


**1. Carregando consulta GCP**

In [6]:
#CRIANDO df
frames = []

#df
df = client.query("""

SELECT
  ano_mes_dia Data,
  EXTRACT(YEAR
  FROM ano_mes_dia) ano,
  EXTRACT(MONTH FROM ano_mes_dia) mes,
  "DIRETO" Rota,
  UPPER(grp_distribuidor) cliente,
  CASE
    WHEN DES_MARCA IS NULL THEN "OUTRAS"
    ELSE UPPER(des_marca)
  END
  des_marca,
  SUM(qtd_vendida) qt_fornecida_umv,
  ROUND(SUM(vlr_preco_consumo),2) vlr_realizado
FROM
  sensitive-refined.sellout.tb_sellout_farma_b2b_rfd
WHERE
  ano_mes_dia >= '2023-01-01'
  AND grp_distribuidor IN ('FARMA CONDE REDE',
    'GRUPO DIMED',
    'GRUPO DPSP',
    'GRUPO DROGARIA ARAUJO',
    'GRUPO INDIANA',
    'GRUPO NISSEI SA',
    'GRUPO PAGUE MENOS',
    'GRUPO RAIA DROGASIL',
    'GRUPO SAO JOAO',
    'GRUPO VENANCIO',
    'GRUPO PANVEL')
GROUP BY
  ALL


""")
df = df.result()
df = df.to_dataframe()


**2. Entendimento dos Dados** </br>
Resumo estatístico e formato dos dados.


In [ ]:
# Mostrar o resumo estatístico do DataFrame
print(df.describe())

In [ ]:
# Mostrar as primeiras linhas do DataFrame
print(df.head())

In [ ]:
# Verificar a existência de valores faltantes
print(df.isnull().sum())

**PROJEÇÕES**

In [7]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings

# Certificar-se de que a coluna de datas está no formato correto
df['Data'] = pd.to_datetime(df['Data'])

# Verificar se a coluna 'rota' existe no DataFrame original
if 'rota' not in df.columns and 'Rota' in df.columns:
    df = df.rename(columns={'Rota': 'rota'})

# Encontrar a data máxima de venda para cada cliente, marca, mês e ano
df_max_data = df.groupby(['cliente', 'mes', 'ano'])['Data'].max().reset_index()
df_max_data = df_max_data.rename(columns={'Data': 'max_data_venda'})

# Fazer o merge da data máxima de venda no DataFrame original
df = pd.merge(df, df_max_data, on=['cliente', 'mes', 'ano'], how='left')

# Calcular os dias passados até a data máxima de venda
df['dias_passados_max'] = (df['max_data_venda'] - df['max_data_venda'].dt.to_period('M').dt.start_time).dt.days + 1

# Calcular o total de dias no mês
df['total_dias_mes'] = df['max_data_venda'].dt.daysinmonth

# Calcular os dias restantes no mês a partir da data máxima de venda
df['dias_restantes_max'] = df['total_dias_mes'] - df['dias_passados_max']

# Agrupar para calcular a soma de vlr_realizado e outras métricas
df_proj = df.groupby(['cliente', 'des_marca', 'mes', 'ano', 'rota']).agg(
    vlr_realizado_total=('vlr_realizado', 'sum'),  # Soma total do valor realizado
    dias_passados_max=('dias_passados_max', 'max'),
    total_dias_mes=('total_dias_mes', 'max'),
    dias_restantes_max=('dias_restantes_max', 'max')
).reset_index()

# --- Cálculo da Projeção Incremental (Curva de Crescimento) ---
# Calcular o acúmulo de vendas diárias até a data máxima de venda para cada cliente, marca, e rota
df['vlr_realizado_acumulado'] = df.groupby(['cliente', 'des_marca', 'mes', 'ano', 'rota'])['vlr_realizado'].cumsum()

# --- Aplicar suavização (usando média móvel de 3 dias, por exemplo) ---
#df['vlr_realizado_suavizado'] = df.groupby(['cliente', 'des_marca', 'mes', 'ano', 'rota'])['vlr_realizado'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Calcular a média móvel de 3 dias para suavizar o impacto de devoluções
df['vlr_realizado_suavizado'] = df.groupby(['cliente', 'des_marca', 'mes', 'ano', 'rota'])['vlr_realizado_acumulado'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Agrupar e selecionar o último valor suavizado por grupo
df_suavizado = df.groupby(['cliente', 'des_marca', 'mes', 'ano', 'rota']).agg(
    vlr_realizado_suavizado=('vlr_realizado_suavizado', 'last')
).reset_index()

# --- Cálculo da taxa de crescimento ---
df_proj['taxa_crescimento'] = (df_suavizado['vlr_realizado_suavizado'] / df_proj['vlr_realizado_total'])


# --- Cálculo da projeção linear ---
df_proj['proj_linear_max'] = (df_proj['vlr_realizado_total'] / df_proj['dias_passados_max']) * df_proj['total_dias_mes']

# --- Cálculo da projeção incremental baseada na taxa de crescimento observada ---
df_proj['proj_incremental'] = df_proj['vlr_realizado_total'] * (1 + df_proj['taxa_crescimento'] * df_proj['dias_restantes_max'] / df_proj['dias_passados_max'])

# Aplicar o modelo Holt-Winters para projeção, verificando se há dados suficientes para sazonalidade
proj_holt_winters = []
for name, group in df.groupby(['cliente', 'des_marca', 'mes', 'ano', 'rota']):
    # Verificar se temos dados suficientes (pelo menos dois ciclos sazonais)
    seasonal_periods = 7  # Exemplo de sazonalidade semanal
    if len(group) >= 2 * seasonal_periods and group['dias_restantes_max'].iloc[0] > 0:
        # Definir um índice de datas adequado e interpolar dados faltantes
        group = group.set_index('Data').asfreq('D')  # Definindo a frequência diária
        group['vlr_realizado'] = group['vlr_realizado'].interpolate(method='linear')  # Interpolar dados faltantes

        # Verificar se há dados suficientes após a interpolação
        if group['vlr_realizado'].isnull().sum() == 0:
            # Aplicar o modelo Holt-Winters com sazonalidade
            hw_model = ExponentialSmoothing(group['vlr_realizado'], trend='add', seasonal='add', seasonal_periods=seasonal_periods).fit()

            # Verificar se dias_restantes é positivo e maior que zero
            dias_restantes = int(group['dias_restantes_max'].iloc[0])
            if dias_restantes > 0:
                # Projeção até o final do mês
                proj = hw_model.forecast(dias_restantes)

                # Somar a projeção total para os dias restantes
                proj_holt_winters.append(proj.sum())
            else:
                proj_holt_winters.append(None)
        else:
            proj_holt_winters.append(None)
    else:
        # Se não houver dados suficientes, rodar o modelo sem sazonalidade
        if len(group) >= 3:
            group = group.set_index('Data').asfreq('D')
            group['vlr_realizado'] = group['vlr_realizado'].interpolate(method='linear')
            hw_model = ExponentialSmoothing(group['vlr_realizado'], trend='add', seasonal=None).fit()

            dias_restantes = int(group['dias_restantes_max'].iloc[0])
            if dias_restantes > 0:
                proj = hw_model.forecast(dias_restantes)
                proj_holt_winters.append(proj.sum())
            else:
                proj_holt_winters.append(None)
        else:
            proj_holt_winters.append(None)

# Adicionar a projeção Holt-Winters ao DataFrame final
df_proj['proj_holt_winters'] = proj_holt_winters

# --- Merge para trazer a Projeção Incremental e Holt-Winters ao DataFrame Final ---
df_final = pd.merge(df_suavizado,
                    df_proj[['cliente', 'des_marca', 'mes', 'ano', 'rota', 'vlr_realizado_total', 'proj_linear_max', 'proj_incremental', 'proj_holt_winters', 'taxa_crescimento', 'dias_passados_max', 'total_dias_mes', 'dias_restantes_max']],
                    on=['cliente', 'des_marca', 'mes', 'ano', 'rota'], how='left')

# Incluir a 'max_data_venda' no df_final sem causar duplicações
df_final = pd.merge(df_final, df_max_data[['cliente', 'mes', 'ano', 'max_data_venda']],
                    on=['cliente', 'mes', 'ano'], how='left')

# Exibir o DataFrame final com as colunas necessárias, incluindo 'max_data_venda'
print(df_final[['cliente', 'des_marca', 'mes', 'ano', 'rota', 'max_data_venda', 'vlr_realizado_suavizado', 'vlr_realizado_total', 'proj_linear_max', 'proj_incremental', 'proj_holt_winters', 'taxa_crescimento', 'dias_passados_max', 'total_dias_mes', 'dias_restantes_max']].head())
warnings.filterwarnings("ignore", message="ConvergenceWarning")



/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_

            cliente        des_marca  mes   ano    rota max_data_venda  \
0  FARMA CONDE REDE  AUSTRALIAN GOLD    1  2023  DIRETO     2023-01-31   
1  FARMA CONDE REDE  AUSTRALIAN GOLD    1  2024  DIRETO     2024-01-31   
2  FARMA CONDE REDE  AUSTRALIAN GOLD    2  2023  DIRETO     2023-02-28   
3  FARMA CONDE REDE  AUSTRALIAN GOLD    2  2024  DIRETO     2024-02-29   
4  FARMA CONDE REDE  AUSTRALIAN GOLD    3  2023  DIRETO     2023-03-31   

   vlr_realizado_suavizado  vlr_realizado_total  proj_linear_max  \
0             44536.020000             45216.27         45216.27   
1             77793.596667             80631.52         80631.52   
2             13792.840000             13830.74         13830.74   
3             56353.726667             57715.42         57715.42   
4             12292.680000             12499.68         12499.68   

   proj_incremental  proj_holt_winters  taxa_crescimento  dias_passados_max  \
0          45216.27                NaN          0.984956           

**Salvar Histórico**

In [9]:
# Definir a data de execução atual
data_execucao = datetime.now().strftime('%Y-%m-%d')

# --- Adicionar a data de execução ao df_final ---
df_final['data_execucao'] = data_execucao  # Adicionando a data de execução ao df_final

# --- Identificar a projeção até o final do mês e o realizado acumulado ---
# Encontrar o valor realizado acumulado até a data atual (última venda)
df_final['realizado_acumulado'] = df.groupby(['cliente', 'des_marca', 'mes', 'ano', 'rota'])['vlr_realizado'].cumsum()

# Calcular o fator de correção
# Primeiro verificamos se já existe histórico para comparação
df_ultima_proj = df_historico[df_historico['data_execucao'] != data_execucao]

if not df_ultima_proj.empty:
    # Comparar o realizado acumulado até a data atual vs. última projeção até a data atual
    df_corr = pd.merge(df_final, df_ultima_proj, on=['cliente', 'des_marca', 'mes', 'ano', 'rota'], suffixes=('_atual', '_anterior'))

    # Calcular o fator de correção
    df_corr['fator_correcao_linear'] = df_corr['realizado_acumulado_atual'] / df_corr['proj_linear_max_anterior']
    df_corr['fator_correcao_incremental'] = df_corr['realizado_acumulado_atual'] / df_corr['proj_incremental_anterior']
    df_corr['fator_correcao_hw'] = df_corr['realizado_acumulado_atual'] / df_corr['proj_holt_winters_anterior']

    # Ajustar as projeções futuras
    df_corr['proj_linear_max_ajustada'] = df_corr['proj_linear_max_atual'] * df_corr['fator_correcao_linear']
    df_corr['proj_incremental_ajustada'] = df_corr['proj_incremental_atual'] * df_corr['fator_correcao_incremental']
    df_corr['proj_holt_winters_ajustada'] = df_corr['proj_holt_winters_atual'] * df_corr['fator_correcao_hw']

    # Incluir as colunas adicionais: 'taxa_crescimento', 'dias_passados_max', 'total_dias_mes', 'dias_restantes_max'
    # Garantir que essas colunas existam no DataFrame final após o merge
    colunas_adicionais = ['taxa_crescimento', 'dias_passados_max', 'total_dias_mes', 'dias_restantes_max']
    for col in colunas_adicionais:
        if f'{col}_atual' not in df_corr.columns:
            df_corr[f'{col}_atual'] = df_corr[col] if col in df_corr.columns else None  # Adicionar valores nulos se a coluna não existir

    # Criar o DataFrame corrigido com as colunas necessárias
    df_final_corrigido = df_corr[['cliente', 'des_marca', 'mes', 'ano', 'rota', 'vlr_realizado_total_atual',
                                  'proj_linear_max_atual', 'proj_incremental_atual', 'proj_holt_winters_atual',
                                  'proj_linear_max_ajustada', 'proj_incremental_ajustada', 'proj_holt_winters_ajustada',
                                  'taxa_crescimento_atual', 'dias_passados_max_atual', 'total_dias_mes_atual',
                                  'dias_restantes_max_atual', 'data_execucao_atual']].copy()

    # Renomear a coluna de 'data_execucao_atual' para 'data_execucao' para consistência
    df_final_corrigido.rename(columns={'data_execucao_atual': 'data_execucao'}, inplace=True)

else:
    # Caso não haja histórico, manter as projeções atuais e copiar para as ajustadas
    df_final_corrigido = df_final.copy()
    df_final_corrigido['proj_linear_max_ajustada'] = df_final_corrigido['proj_linear_max']
    df_final_corrigido['proj_incremental_ajustada'] = df_final_corrigido['proj_incremental']
    df_final_corrigido['proj_holt_winters_ajustada'] = df_final_corrigido['proj_holt_winters']

    # Adicionar as colunas que estavam faltando com valores nulos
    df_final_corrigido['taxa_crescimento_atual'] = df_final['taxa_crescimento'] if 'taxa_crescimento' in df_final.columns else None
    df_final_corrigido['dias_passados_max_atual'] = df_final['dias_passados_max'] if 'dias_passados_max' in df_final.columns else None
    df_final_corrigido['total_dias_mes_atual'] = df_final['total_dias_mes'] if 'total_dias_mes' in df_final.columns else None
    df_final_corrigido['dias_restantes_max_atual'] = df_final['dias_restantes_max'] if 'dias_restantes_max' in df_final.columns else None

# --- Remover qualquer linha do histórico que já tenha a data de execução atual ---
df_historico = df_historico[df_historico['data_execucao'] != data_execucao]

# --- Concatenar o novo resultado ao histórico atualizado ---
df_historico = pd.concat([df_historico, df_final_corrigido])

# --- Salvar o histórico atualizado ---
with open(historico_file, 'wb') as f:
    pickle.dump(df_historico, f)

print(f"Histórico atualizado e salvo com sucesso em {historico_file}.")

# Exibir o DataFrame final corrigido
print(df_final_corrigido[['cliente', 'des_marca', 'mes', 'ano', 'rota', 'proj_linear_max', 'proj_linear_max_ajustada',
                          'proj_incremental', 'proj_incremental_ajustada', 'proj_holt_winters', 'proj_holt_winters_ajustada',
                          'taxa_crescimento_atual', 'dias_passados_max_atual', 'total_dias_mes_atual', 'dias_restantes_max_atual']].head())


Histórico atualizado e salvo com sucesso em /content/drive/MyDrive/Forecast/historico_projecoes.pkl.
            cliente        des_marca  mes   ano    rota  proj_linear_max  \
0  FARMA CONDE REDE  AUSTRALIAN GOLD    1  2023  DIRETO         45216.27   
1  FARMA CONDE REDE  AUSTRALIAN GOLD    1  2024  DIRETO         80631.52   
2  FARMA CONDE REDE  AUSTRALIAN GOLD    2  2023  DIRETO         13830.74   
3  FARMA CONDE REDE  AUSTRALIAN GOLD    2  2024  DIRETO         57715.42   
4  FARMA CONDE REDE  AUSTRALIAN GOLD    3  2023  DIRETO         12499.68   

   proj_linear_max_ajustada  proj_incremental  proj_incremental_ajustada  \
0                  45216.27          45216.27                   45216.27   
1                  80631.52          80631.52                   80631.52   
2                  13830.74          13830.74                   13830.74   
3                  57715.42          57715.42                   57715.42   
4                  12499.68          12499.68                 

In [10]:
# Carregar o histórico salvo para verificação
with open(historico_file, 'rb') as f:
    df_historico = pickle.load(f)

# Verificar as primeiras linhas do DataFrame histórico
print(df_historico.head())


            cliente        des_marca  mes   ano    rota  \
0  FARMA CONDE REDE  AUSTRALIAN GOLD    1  2023  DIRETO   
1  FARMA CONDE REDE  AUSTRALIAN GOLD    1  2024  DIRETO   
2  FARMA CONDE REDE  AUSTRALIAN GOLD    2  2023  DIRETO   
3  FARMA CONDE REDE  AUSTRALIAN GOLD    2  2024  DIRETO   
4  FARMA CONDE REDE  AUSTRALIAN GOLD    3  2023  DIRETO   

   vlr_realizado_suavizado  vlr_realizado_total  proj_linear_max  \
0             44536.020000             45216.27         45216.27   
1             77793.596667             80631.52         80631.52   
2             13792.840000             13830.74         13830.74   
3             56353.726667             57715.42         57715.42   
4             12292.680000             12499.68         12499.68   

   proj_incremental  proj_holt_winters  taxa_crescimento  dias_passados_max  \
0          45216.27                NaN          0.984956                 31   
1          80631.52                NaN          0.964804                 31   

**SALVAR NO GCP**

In [10]:
import pandas as pd
from google.cloud import bigquery
import time

# Configurações do BigQuery
sandbox = 'ssbx-revops'
bd_tmp = 'ssbx-revops.Forecast.tb_fcst_sellout_direto_hw'
tabela_nova = bd_tmp

# Inicializa o cliente BigQuery
client = bigquery.Client(project=sandbox, location='US')

# Define o nome da nova tabela
tabela_nova = bd_tmp

# Converte a matriz df para um DataFrame pandas (supondo que bd2 seja a matriz a ser convertida)
tab_nova = pd.DataFrame(df_final_corrigido)  # Use df diretamente se for um DataFrame

# Configuração do carregamento
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition='WRITE_TRUNCATE'  # Substitui os dados existentes
)

# Marca o início do tempo
tempo_inicial = time.time()

# Carregando os dados para o BigQuery
job = client.load_table_from_dataframe(tab_nova, tabela_nova, job_config=job_config)
job.result()  # Aguarda a conclusão do job

# Marca o tempo final
tempo_final = time.time()

# Calcula o tempo decorrido
tempo_decorrido = tempo_final - tempo_inicial

# Formata o tempo decorrido para hh:mm:ss
tempo_formatado = time.strftime("%H:%M:%S", time.gmtime(tempo_decorrido))

print(f'Tabela {tabela_nova} foi carregada no BigQuery com sucesso')
print(f'Tempo decorrido: {tempo_formatado}')


Tabela ssbx-revops.Forecast.tb_fcst_sellout_direto_hw foi carregada no BigQuery com sucesso
Tempo decorrido: 00:00:05


In [11]:
print(df.index.duplicated().sum())  # Verifica se há índices duplicados


0
